In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from groclient import GroClient
import os

In [2]:
# setup client
API_HOST = 'api.gro-intelligence.com'
GROAPI_TOKEN = os.environ['GROAPI_TOKEN']
client = GroClient(API_HOST, GROAPI_TOKEN)

# Gro Supply & Demand - Soybeans

We want to recreate the balance sheets for soybeans on the portal using the API.

For each country, we want to pull relevant data from the API:
* Supply
* Demand
* Stocks

In the portal, we look at the US, Brazil, China, and Argentina. For each line of the balance sheet, we call a separate metric-item pair. (e.g. the line in the US balance sheet for soy crush involves querying from the Gro API with metric = crushing (mass) and item = soybeans).

------------------

Starting with a simple example: annual US Soy Crush from PS&D in bushels.

In order to get this data series, we need to write a query with the correct metric, item, region, and source.

In [3]:
us_soy_crush_query = {
    'metric_id': 1800032, # metric ID for crushing (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for USDA PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20 # unit ID for bushels
}

# now can use the API client to pull this data series
us_soy_crush = pd.DataFrame(client.get_data_points(**us_soy_crush_query))
us_soy_crush

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,4.789948e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.374912e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,5.594274e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,5.764031e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,6.059086e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
5,1969-09-01T00:00:00.000Z,1970-08-31T00:00:00.000Z,7.373051e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
6,1970-09-01T00:00:00.000Z,1971-08-31T00:00:00.000Z,7.601231e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
7,1971-09-01T00:00:00.000Z,1972-08-31T00:00:00.000Z,7.205498e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
8,1972-09-01T00:00:00.000Z,1973-08-31T00:00:00.000Z,7.217991e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
9,1973-09-01T00:00:00.000Z,1974-08-31T00:00:00.000Z,8.213386e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9


We can also limit the dates of our query. Suppose we only care about the annual US soy crush for this year.

In [4]:
us_soy_crush_query_dates = {
    'metric_id': 1800032, # metric ID for crushing (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
    'start_date': '2022-01-01', # start of the year
    'end_date': '2022-12-31' # end of the year
}

# now can use the API client to pull this data series
us_soy_crush_dates = pd.DataFrame(client.get_data_points(**us_soy_crush_query_dates))
us_soy_crush_dates

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,2021-09-01T00:00:00.000Z,2022-08-31T00:00:00.000Z,2.215002e+09,20,{},14,1,2022-03-09T00:00:00.000Z,2022-03-09T00:00:00.000Z,1800032,270,1215,0,9


## Total Supply, Total Demand, Ending Stocks-to-Use

In the balance sheets, we have annual values for total supply, total demand, and ending stocks-to-use. 

The total supply is from USDA PS&D, the total demand number is calculated as total consumption + exports, and ending stocks-to-use is calcualted as ending carryover / total demand.

We can write some queries and do some simple calculations to get these numbers.

In [5]:
# total supply
# we are still pulling data with item soybeans, region US, and source PS&D
# just change the metric_id
us_soy_total_supply_query = {
    'metric_id': 1880032, # metric ID for supply quantity (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

us_soy_total_supply = pd.DataFrame(client.get_data_points(**us_soy_total_supply_query))
us_soy_total_supply

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,7.682436e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,8.753156e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,9.641258e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,1.066605e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,1.273327e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
5,1969-09-01T00:00:00.000Z,1970-08-31T00:00:00.000Z,1.459986e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
6,1970-09-01T00:00:00.000Z,1971-08-31T00:00:00.000Z,1.356956e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
7,1971-09-01T00:00:00.000Z,1972-08-31T00:00:00.000Z,1.274907e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
8,1972-09-01T00:00:00.000Z,1973-08-31T00:00:00.000Z,1.342589e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
9,1973-09-01T00:00:00.000Z,1974-08-31T00:00:00.000Z,1.607219e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9


In [6]:
# total demand
# need to do separate queries for total consumption and exports

# Domestic consumption from PS&D
us_soy_consumption_query = {
    'metric_id': 1480032, # metric ID for domestic consumption (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

# Annual export volume from PS&D
us_soy_exports_query = {
    'metric_id': 20032, # metric ID for export volume (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

us_soy_consumption = pd.DataFrame(client.get_data_points(**us_soy_consumption_query))
us_soy_exports = pd.DataFrame(client.get_data_points(**us_soy_exports_query))

display(us_soy_consumption.head())
display(us_soy_exports.head())

# get total demand but adding consumption and exports
us_soy_demand = pd.merge(
    us_soy_consumption[['start_date', 'end_date', 'value']].rename(columns={'value': 'consumption'}),
    us_soy_exports[['start_date', 'end_date', 'value']].rename(columns={'value': 'exports'}),
    on=['start_date', 'end_date']
)
us_soy_demand['demand'] = us_soy_demand['consumption'].add(us_soy_demand['exports'])
us_soy_demand.tail()

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,5.263945e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.890798e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,6.124122e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,6.336870e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,6.597018e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,2.121599e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,2.505941e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,2.615805e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,2.665777e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,2.867869e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9


,start_date,end_date,consumption,exports,demand
53,2017-09-01T00:00:00.000Z,2018-08-31T00:00:00.000Z,2.163230e+09,2.133761e+09,4.296990e+09
54,2018-09-01T00:00:00.000Z,2019-08-31T00:00:00.000Z,2.217868e+09,1.753460e+09,3.971328e+09
55,2019-09-01T00:00:00.000Z,2020-08-31T00:00:00.000Z,2.272580e+09,1.679238e+09,3.951817e+09
56,2020-09-01T00:00:00.000Z,2021-08-31T00:00:00.000Z,2.243184e+09,2.260564e+09,4.503749e+09
57,2021-09-01T00:00:00.000Z,2022-08-31T00:00:00.000Z,2.331884e+09,2.089999e+09,4.421883e+09


In [7]:
# ending stocks-to-use
# calculate ending stocks-to-use = ending carryover / total demand = (total supply - total demand) / total_demand

us_soy_stocks_to_use = pd.merge(
    us_soy_total_supply[['start_date', 'end_date', 'value']].rename(columns={'value': 'supply'}),
    us_soy_demand[['start_date', 'end_date', 'demand']],
    on=['start_date', 'end_date']
)

us_soy_stocks_to_use['carryover'] = us_soy_stocks_to_use['supply'].subtract(us_soy_stocks_to_use['demand'])
us_soy_stocks_to_use['stocks_to_use'] = us_soy_stocks_to_use['carryover'].divide(us_soy_stocks_to_use['demand'])

us_soy_stocks_to_use

,start_date,end_date,supply,demand,carryover,stocks_to_use
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,7.682436e+08,7.385544e+08,2.968915e+07,0.040199
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,8.753156e+08,8.396739e+08,3.564168e+07,0.042447
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,9.641258e+08,8.739928e+08,9.013303e+07,0.103128
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,1.066605e+09,9.002647e+08,1.663401e+08,0.184768
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,1.273327e+09,9.464887e+08,3.268379e+08,0.345316
5,1969-09-01T00:00:00.000Z,1970-08-31T00:00:00.000Z,1.459986e+09,1.230152e+09,2.298337e+08,0.186834
6,1970-09-01T00:00:00.000Z,1971-08-31T00:00:00.000Z,1.356956e+09,1.258188e+09,9.876787e+07,0.078500
7,1971-09-01T00:00:00.000Z,1972-08-31T00:00:00.000Z,1.274907e+09,1.202962e+09,7.194475e+07,0.059806
8,1972-09-01T00:00:00.000Z,1973-08-31T00:00:00.000Z,1.342589e+09,1.282954e+09,5.963551e+07,0.046483
9,1973-09-01T00:00:00.000Z,1974-08-31T00:00:00.000Z,1.607219e+09,1.436470e+09,1.707494e+08,0.118867


## Querying multiple metrics at once

The queries we've been writing have been similar (same item, region, source). Can also query multiple metrics at once.

Query for supply, exports, and consumption at all once.

Take these three queries and make into one:
```
# Total Supply from PS&D
us_soy_total_supply_query = {
    'metric_id': 1880032, # metric ID for supply quantity (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

# Domestic consumption from PS&D
us_soy_consumption_query = {
    'metric_id': 1480032, # metric ID for domestic consumption (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

# Annual export volume from PS&D
us_soy_exports_query = {
    'metric_id': 20032, # metric ID for export volume (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}
```

In [8]:
combined_query = {
    'metric_id': [1880032, 1480032, 20032], # supply, domestic consumption, export
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for US PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
}

us_soy_combined = pd.DataFrame(client.get_data_points(**combined_query))
us_soy_combined

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,5.263945e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.890798e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,6.124122e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,6.336870e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,6.597018e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2017-09-01T00:00:00.000Z,2018-08-31T00:00:00.000Z,4.735089e+09,20,{},14,1,2020-09-11T00:00:00.000Z,2020-09-11T00:00:00.000Z,1880032,270,1215,0,9
170,2018-09-01T00:00:00.000Z,2019-08-31T00:00:00.000Z,4.880375e+09,20,{},14,1,2021-07-12T00:00:00.000Z,2021-07-12T00:00:00.000Z,1880032,270,1215,0,9
171,2019-09-01T00:00:00.000Z,2020-08-31T00:00:00.000Z,4.476375e+09,20,{},14,1,2021-07-12T00:00:00.000Z,2021-07-12T00:00:00.000Z,1880032,270,1215,0,9
172,2020-09-01T00:00:00.000Z,2021-08-31T00:00:00.000Z,4.760736e+09,20,{},14,1,2022-02-09T00:00:00.000Z,2022-02-09T00:00:00.000Z,1880032,270,1215,0,9


In [9]:
just_supply = us_soy_combined[us_soy_combined['metric_id'] == 1880032]
just_supply.head()

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
116,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,7.682436e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
117,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,8.753156e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
118,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,9.641258e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
119,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,1.066605e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9
120,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,1.273327e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1880032,270,1215,0,9


## Gro Yield Model

The Gro Yield Model ouputs predictions for crop yield. 

We can query the predictions from the API:

In [10]:
us_soy_gro_yield_query = {
    'metric_id': 170037, # yield (mass/area)
    'item_id': 270, # soybeans
    'region_id': 1215, # US
    'source_id': 32, # Gro Yield Model
    'frequency_id': 9, # annual
    'unit_id': 287 # bushel per acre
}
us_soy_gro_yield = pd.DataFrame(client.get_data_points(**us_soy_gro_yield_query))
us_soy_gro_yield

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,2009-01-01T00:00:00.000Z,2009-12-31T00:00:00.000Z,46.076001,287,{},287,1,2009-10-31T00:00:00.000Z,2020-08-03T20:50:16.000Z,170037,270,1215,0,9
1,2010-01-01T00:00:00.000Z,2010-12-31T00:00:00.000Z,44.734604,287,{},287,1,2010-10-31T00:00:00.000Z,2020-08-03T20:54:17.000Z,170037,270,1215,0,9
2,2011-01-01T00:00:00.000Z,2011-12-31T00:00:00.000Z,43.907566,287,{},287,1,2011-10-31T00:00:00.000Z,2020-08-03T20:57:09.000Z,170037,270,1215,0,9
3,2012-01-01T00:00:00.000Z,2012-12-31T00:00:00.000Z,37.277818,287,{},287,1,2012-10-30T00:00:00.000Z,2020-08-03T20:57:32.000Z,170037,270,1215,0,9
4,2013-01-01T00:00:00.000Z,2013-12-31T00:00:00.000Z,44.581517,287,{},287,1,2013-10-31T00:00:00.000Z,2020-04-22T00:00:00.000Z,170037,270,1215,0,9
5,2014-01-01T00:00:00.000Z,2014-12-31T00:00:00.000Z,47.729571,287,{},287,1,2014-10-31T00:00:00.000Z,2020-04-22T00:00:00.000Z,170037,270,1215,0,9
6,2015-01-01T00:00:00.000Z,2015-12-31T00:00:00.000Z,47.623592,287,{},287,1,2015-10-31T00:00:00.000Z,2020-04-22T00:00:00.000Z,170037,270,1215,0,9
7,2016-01-01T00:00:00.000Z,2016-12-31T00:00:00.000Z,49.801396,287,{},287,1,2016-10-30T00:00:00.000Z,2020-04-22T00:00:00.000Z,170037,270,1215,0,9
8,2017-01-01T00:00:00.000Z,2017-12-31T00:00:00.000Z,49.713726,287,{},287,1,2017-10-31T00:00:00.000Z,2020-04-22T00:00:00.000Z,170037,270,1215,0,9
9,2018-01-01T00:00:00.000Z,2018-12-31T00:00:00.000Z,50.610125,287,{},287,1,2019-01-17T00:00:00.000Z,2019-01-17T00:00:00.000Z,170037,270,1215,0,9


Suppose we want this at a smaller granularity. We can use the same query with a different region ID.

Doing the same query but for individual states.

In [11]:
us_states = client.get_descendant_regions(1215, descendant_level=4) # level 4 is state for US
print('Example of US state dictionary', us_states[25])

us_states_region_id = [x['id'] for x in us_states]
print('Example of just the region ID', us_states_region_id[25])

us_soy_gro_yield_states = us_soy_gro_yield_query.copy()
us_soy_gro_yield_states['region_id'] = us_states_region_id
pd.DataFrame(client.get_data_points(**us_soy_gro_yield_states))

Example of US state dictionary {'aliases': ['MO', 'US-MO'], 'contains': [138295, 138320, 138283, 138280, 138340, 138282, 138281, 138265, 138261, 138304, 138341, 138277, 138257, 138314, 138246, 138317, 138292, 138287, 138253, 138335, 138312, 138347, 138270, 138275, 138264, 138332, 138326, 138274, 138302, 138339, 138250, 138331, 138328, 138299, 138293, 138291, 138276, 138258, 138298, 138309, 138271, 138266, 138247, 138249, 138343, 138239, 138238, 138325, 138242, 138289, 138279, 138311, 138268, 138350, 138244, 138296, 138262, 138303, 138315, 138327, 138336, 138260, 138267, 138245, 138310, 138330, 138288, 138323, 138338, 1001233, 138333, 138273, 138346, 138337, 138259, 138254, 138263, 138297, 138278, 138349, 138255, 138248, 138308, 138345, 138256, 138344, 138301, 138319, 138342, 138321, 138322, 138329, 138313, 138240, 138306, 138243, 138318, 138251, 138272, 138307, 138241, 138334, 138305, 138285, 138294, 138300, 1001241, 138269, 138351, 138252, 138324, 138286, 138316, 138348, 138284, 13829

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,2018-01-01T00:00:00.000Z,2018-12-31T00:00:00.000Z,48.289228,287,{},287,1,2019-01-17T00:00:00.000Z,2019-01-17T00:00:00.000Z,170037,270,13054,0,9
1,2019-01-01T00:00:00.000Z,2019-12-31T00:00:00.000Z,49.133392,287,{},287,1,2019-12-31T00:00:00.000Z,2019-12-31T00:00:00.000Z,170037,270,13054,0,9
2,2020-01-01T00:00:00.000Z,2020-12-31T00:00:00.000Z,51.999516,287,{},287,1,2020-12-31T00:00:00.000Z,2020-12-31T05:03:40.000Z,170037,270,13054,0,9
3,2021-01-01T00:00:00.000Z,2021-12-31T00:00:00.000Z,51.771675,287,{},287,1,2021-12-31T00:00:00.000Z,2021-12-31T02:27:41.000Z,170037,270,13054,0,9
4,2018-01-01T00:00:00.000Z,2018-12-31T00:00:00.000Z,59.403193,287,{},287,1,2019-01-17T00:00:00.000Z,2019-01-17T00:00:00.000Z,170037,270,13064,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2021-01-01T00:00:00.000Z,2021-12-31T00:00:00.000Z,51.159862,287,{},287,1,2021-12-31T00:00:00.000Z,2021-12-31T02:27:41.000Z,170037,270,13093,0,9
68,2018-01-01T00:00:00.000Z,2018-12-31T00:00:00.000Z,51.227911,287,{},287,1,2019-01-17T00:00:00.000Z,2019-01-17T00:00:00.000Z,170037,270,13100,0,9
69,2019-01-01T00:00:00.000Z,2019-12-31T00:00:00.000Z,47.721321,287,{},287,1,2019-12-31T00:00:00.000Z,2019-12-31T00:00:00.000Z,170037,270,13100,0,9
70,2020-01-01T00:00:00.000Z,2020-12-31T00:00:00.000Z,53.881570,287,{},287,1,2020-12-31T00:00:00.000Z,2020-12-31T05:03:40.000Z,170037,270,13100,0,9


## Putting this all together: US soybeans supply

Suppose we want to get all the values used in the supply part of the US soybeans balance sheet.

Metric IDs we care about:
* Supply
    * Yield - 170037
        * Source for this is Gro Yield Model (source ID 32)
    * Planted Area - 2580001
        * Source for this is WASDE (source ID 106)
    * Harvested Area - 570001
        * Source for this is USDA PS&D (source ID 14)
    * Production - 860032
        * Source for this is USDA PS&D (source ID 14)
    * Imports - 30032
        * Source for this is USDA PS&D (source ID 14)
        
We already have loaded in the data series for yield, so we can write queries for planted area, harvested area, production, and imports.

In [12]:
us_soy_area_query = {
    'item_id': 270, # soybeans
    'region_id': 1215, # US
    'frequency_id': 9, # annual
    'unit_id': 41 # acre
}

# PLANTED AREA
us_soy_planted_area_query = us_soy_area_query.copy()
us_soy_planted_area_query['metric_id'] = 2580001 # area planted
us_soy_planted_area_query['source_id'] = 106 # WASDE

us_soy_planted_area = pd.DataFrame(client.get_data_points(**us_soy_planted_area_query))
print('US Soybeans Planted Area')
display(us_soy_planted_area.head())

# HARVESTED AREA
us_soy_harvest_area_query = us_soy_area_query.copy()
us_soy_harvest_area_query['metric_id'] = 570001 # area harvested
us_soy_harvest_area_query['source_id'] = 14 # USDA PS&D

us_soy_harvest_area = pd.DataFrame(client.get_data_points(**us_soy_harvest_area_query))
print('US Soybeans Harvested Area')
display(us_soy_harvest_area.head())

### PRODUCTION AND IMPORTS
us_soy_psd_query = {
    'item_id': 270, # soybeans
    'region_id': 1215, # US
    'frequency_id': 9, # annual
    'source_id': 14, # USDA PS&D
    'unit_id': 20 # bushel
}

# PRODUCTION
us_soy_production_query = us_soy_psd_query.copy()
us_soy_production_query['metric_id'] = 860032 # production quantity (mass)

us_soy_production = pd.DataFrame(client.get_data_points(**us_soy_production_query))
print('US Soybeans Production')
display(us_soy_production.head())

# IMPORTS
us_soy_import_query = us_soy_psd_query.copy()
us_soy_import_query['metric_id'] = 30032 # import volume (mass)

us_soy_import = pd.DataFrame(client.get_data_points(**us_soy_import_query))
print('US Soybeans Imports')
display(us_soy_import.head())

US Soybeans Planted Area


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,2008-09-01T00:00:00.000Z,2009-08-31T00:00:00.000Z,75700000,41,{},41,1,2011-04-08T00:00:00.000Z,2019-03-29T00:00:00.000Z,2580001,270,1215,0,9
1,2009-09-01T00:00:00.000Z,2010-08-31T00:00:00.000Z,77500000,41,{},41,1,2012-04-10T00:00:00.000Z,2019-03-29T00:00:00.000Z,2580001,270,1215,0,9
2,2010-09-01T00:00:00.000Z,2011-08-31T00:00:00.000Z,77400000,41,{},41,1,2013-04-10T00:00:00.000Z,2019-03-29T00:00:00.000Z,2580001,270,1215,0,9
3,2011-09-01T00:00:00.000Z,2012-08-31T00:00:00.000Z,75000000,41,{},41,1,2014-04-09T00:00:00.000Z,2019-03-29T00:00:00.000Z,2580001,270,1215,0,9
4,2012-09-01T00:00:00.000Z,2013-08-31T00:00:00.000Z,77200000,41,{},41,1,2015-04-09T00:00:00.000Z,2019-03-29T00:00:00.000Z,2580001,270,1215,0,9


US Soybeans Harvested Area


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,3.079427e+07,41,{},42,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,570001,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,3.444896e+07,41,{},42,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,570001,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,3.654689e+07,41,{},42,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,570001,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,3.980621e+07,41,{},42,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,570001,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,4.139262e+07,41,{},42,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,570001,270,1215,0,9


US Soybeans Production


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,7.009285e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,860032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,8.456264e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,860032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,9.284841e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,860032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,9.764718e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,860032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,1.106986e+09,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,860032,270,1215,0,9


US Soybeans Imports


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,0.0,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,30032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,0.0,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,30032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,0.0,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,30032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,0.0,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,30032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,0.0,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,30032,270,1215,0,9


In [13]:
# putting all supply together
def get_end_year(df):
    df['end_year'] = pd.DatetimeIndex(df['end_date']).year
    return df

def get_end_year_yield(df):
    '''
    The Gro Yield model works on a different year, not the market year. We account for year alignment here.
    '''
    df['end_year'] = pd.DatetimeIndex(df['end_date']).year + 1
    return df

us_soy_supply = pd.merge(
    get_end_year_yield(us_soy_gro_yield)[['end_year', 'value']].rename(columns={'value': 'yield'}),
    get_end_year(us_soy_planted_area)[['end_year', 'value']].rename(columns={'value': 'planted_area'}),
    on='end_year'
).merge(
    get_end_year(us_soy_harvest_area)[['end_year', 'value']].rename(columns={'value': 'harvested_area'}),
    on='end_year'
).merge(
    get_end_year(us_soy_production)[['end_year', 'value']].rename(columns={'value': 'production'}),
    on='end_year'
).merge(
    get_end_year(us_soy_import)[['end_year', 'value']].rename(columns={'value': 'import'}),
    on='end_year'
)
display(us_soy_supply)
display(us_soy_supply.pivot_table(columns='end_year'))

,end_year,yield,planted_area,harvested_area,production,import
0,2010,46.076001,77500000,7.637286e+07,3.360974e+09,1.458737e+07
1,2011,44.734604,77400000,7.661008e+07,3.331321e+09,1.444039e+07
2,2012,43.907566,75000000,7.377578e+07,3.097189e+09,1.613062e+07
3,2013,37.277818,77200000,7.614305e+07,3.042073e+09,4.052863e+07
4,2014,44.581517,76800000,7.623201e+07,3.357042e+09,7.176103e+07
5,2015,47.729571,83300000,8.260980e+07,3.928117e+09,3.321658e+07
6,2016,47.623592,82700000,8.174246e+07,3.926795e+09,2.355290e+07
7,2017,49.801396,83500000,8.270617e+07,4.296513e+09,2.226686e+07
8,2018,49.713726,90200000,8.954111e+07,4.411668e+09,2.182594e+07
9,2019,50.610125,89200000,8.759392e+07,4.428203e+09,1.407295e+07


end_year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
harvested_area,7.637286e+07,7.661008e+07,7.377578e+07,7.614305e+07,7.623201e+07,8.260980e+07,8.174246e+07,8.270617e+07,8.954111e+07,8.759392e+07,7.493965e+07,8.260239e+07,8.633121e+07
import,1.458737e+07,1.444039e+07,1.613062e+07,4.052863e+07,7.176103e+07,3.321658e+07,2.355290e+07,2.226686e+07,2.182594e+07,1.407295e+07,1.539574e+07,1.984176e+07,1.499155e+07
planted_area,7.750000e+07,7.740000e+07,7.500000e+07,7.720000e+07,7.680000e+07,8.330000e+07,8.270000e+07,8.350000e+07,9.020000e+07,8.920000e+07,7.610000e+07,8.340000e+07,8.720000e+07
production,3.360974e+09,3.331321e+09,3.097189e+09,3.042073e+09,3.357042e+09,3.928117e+09,3.926795e+09,4.296513e+09,4.411668e+09,4.428203e+09,3.551932e+09,4.216337e+09,4.435258e+09
yield,4.607600e+01,4.473460e+01,4.390757e+01,3.727782e+01,4.458152e+01,4.772957e+01,4.762359e+01,4.980140e+01,4.971373e+01,5.061013e+01,4.664752e+01,5.144973e+01,5.111652e+01


We are looking at limited years because WASDE planted area only goes back until 2009. Could merge differently to get all past years.

## Demand

Just like we put together the supply side of the balance sheet, we can put together the demand side as well.
Metric IDs we care about:
* Demand
    * Crush - 1800032
        * Source for this is USDA PS&D (source ID 14)
    * Waste and Feed - 1760032
        * Source for this is USDA PS&D (source ID 14)
    * Domestic Consumption - 1480032
        * Source for this is USDA PS&D (source ID 14)
    * Exports - 20032
        * Source for this is USDA PS&D (source ID 14)

In [14]:
# CRUSH
us_soy_crush_query = us_soy_psd_query.copy()
us_soy_crush_query['metric_id'] = 1800032 # Crushing (mass)

us_soy_crush = pd.DataFrame(client.get_data_points(**us_soy_crush_query))
print('US Soybeans Crush')
display(us_soy_crush.head())

# WASTE AND FEED
us_soy_feed_query = us_soy_psd_query.copy()
us_soy_feed_query['metric_id'] = 1760032 # Feed Use (mass)

us_soy_feed = pd.DataFrame(client.get_data_points(**us_soy_feed_query))
print('US Soybeans Feed and Waste')
display(us_soy_feed.head())

# DOMESTIC CONSUMPTION
us_soy_consumption_query = us_soy_psd_query.copy()
us_soy_consumption_query['metric_id'] = 1480032 # Domestic Consumption (mass)

us_soy_consumption = pd.DataFrame(client.get_data_points(**us_soy_consumption_query))
print('US Soybeans Domestic Consumption')
display(us_soy_consumption.head())

# EXPORTS
us_soy_exports_query = us_soy_psd_query.copy()
us_soy_exports_query['metric_id'] = 20032 # Export Volume (mass)

us_soy_exports = pd.DataFrame(client.get_data_points(**us_soy_exports_query))
print('US Soybeans Export Volume')
display(us_soy_exports.head())

US Soybeans Crush


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,4.789948e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.374912e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,5.594274e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,5.764031e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,6.059086e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1800032,270,1215,0,9


US Soybeans Feed and Waste


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,4.739976e+07,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1760032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.158858e+07,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1760032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,5.298485e+07,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1760032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,5.728390e+07,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1760032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,5.379322e+07,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1760032,270,1215,0,9


US Soybeans Domestic Consumption


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,5.263945e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,5.890798e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,6.124122e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,6.336870e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,6.597018e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,1480032,270,1215,0,9


US Soybeans Export Volume


,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,1964-09-01T00:00:00.000Z,1965-08-31T00:00:00.000Z,2.121599e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
1,1965-09-01T00:00:00.000Z,1966-08-31T00:00:00.000Z,2.505941e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
2,1966-09-01T00:00:00.000Z,1967-08-31T00:00:00.000Z,2.615805e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
3,1967-09-01T00:00:00.000Z,1968-08-31T00:00:00.000Z,2.665777e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9
4,1968-09-01T00:00:00.000Z,1969-08-31T00:00:00.000Z,2.867869e+08,20,{},14,1,2006-06-09T00:00:00.000Z,2020-11-02T00:00:00.000Z,20032,270,1215,0,9


In [15]:
# putting all demand together
us_soy_demand = pd.merge(
    get_end_year(us_soy_crush)[['end_year', 'value']].rename(columns={'value': 'crush'}),
    get_end_year(us_soy_feed)[['end_year', 'value']].rename(columns={'value': 'feed_use'}),
    on='end_year'
).merge(
    get_end_year(us_soy_consumption)[['end_year', 'value']].rename(columns={'value': 'consumption'}),
    on='end_year'
).merge(
    get_end_year(us_soy_exports)[['end_year', 'value']].rename(columns={'value': 'export_vol'}),
    on='end_year'
)
us_soy_demand['total_demand'] = us_soy_demand['consumption'].add(us_soy_demand['export_vol'])
display(us_soy_demand)
display(us_soy_demand.pivot_table(columns='end_year'))

,end_year,crush,feed_use,consumption,export_vol,total_demand
0,1965,4.789948e+08,4.739976e+07,5.263945e+08,2.121599e+08,7.385544e+08
1,1966,5.374912e+08,5.158858e+07,5.890798e+08,2.505941e+08,8.396739e+08
2,1967,5.594274e+08,5.298485e+07,6.124122e+08,2.615805e+08,8.739928e+08
3,1968,5.764031e+08,5.728390e+07,6.336870e+08,2.665777e+08,9.002647e+08
4,1969,6.059086e+08,5.379322e+07,6.597018e+08,2.867869e+08,9.464887e+08
5,1970,7.373051e+08,6.026016e+07,7.975653e+08,4.325871e+08,1.230152e+09
6,1971,7.601231e+08,6.426526e+07,8.243884e+08,4.337997e+08,1.258188e+09
7,1972,7.205498e+08,6.558804e+07,7.861379e+08,4.168239e+08,1.202962e+09
8,1973,7.217991e+08,8.171866e+07,8.035178e+08,4.794357e+08,1.282954e+09
9,1974,8.213386e+08,7.598659e+07,8.973252e+08,5.391447e+08,1.436470e+09


end_year,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
consumption,5.263945e+08,5.890798e+08,6.124122e+08,6.336870e+08,6.597018e+08,7.975653e+08,8.243884e+08,7.861379e+08,8.035178e+08,8.973252e+08,...,1.783921e+09,1.838780e+09,2.020516e+09,2.001629e+09,2.047339e+09,2.163230e+09,2.217868e+09,2.272580e+09,2.243184e+09,2.331884e+09
crush,4.789948e+08,5.374912e+08,5.594274e+08,5.764031e+08,6.059086e+08,7.373051e+08,7.601231e+08,7.205498e+08,7.217991e+08,8.213386e+08,...,1.689011e+09,1.734023e+09,1.873025e+09,1.886253e+09,1.901208e+09,2.054945e+09,2.092020e+09,2.164589e+09,2.140595e+09,2.215002e+09
export_vol,2.121599e+08,2.505941e+08,2.615805e+08,2.665777e+08,2.867869e+08,4.325871e+08,4.337997e+08,4.168239e+08,4.794357e+08,5.391447e+08,...,1.327524e+09,1.638562e+09,1.842197e+09,1.942619e+09,2.166573e+09,2.133761e+09,1.753460e+09,1.679238e+09,2.260564e+09,2.089999e+09
feed_use,4.739976e+07,5.158858e+07,5.298485e+07,5.728390e+07,5.379322e+07,6.026016e+07,6.426526e+07,6.558804e+07,8.171866e+07,7.598659e+07,...,9.490975e+07,1.047571e+08,1.474904e+08,1.153762e+08,1.461309e+08,1.082846e+08,1.258482e+08,1.079906e+08,1.025892e+08,1.168827e+08
total_demand,7.385544e+08,8.396739e+08,8.739928e+08,9.002647e+08,9.464887e+08,1.230152e+09,1.258188e+09,1.202962e+09,1.282954e+09,1.436470e+09,...,3.111445e+09,3.477342e+09,3.862713e+09,3.944248e+09,4.213912e+09,4.296990e+09,3.971328e+09,3.951817e+09,4.503749e+09,4.421883e+09


## Querying data from other regions

We also care about China, Brazil, and Argentina. Can replace the region IDs in all the queries with the region IDs for these countries.

Let's get soy crush from the US (1215), Brazil (1029), China (1231), and Argentina (1010) for example. Let's limit it to this year only.

Refer to our query for US soy crush:
```
us_soy_crush_query = {
    'metric_id': 1800032, # metric ID for crushing (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': 1215, # region ID for the US
    'source_id': 14, # source ID for USDA PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20 # unit ID for bushels
}
```

In [16]:
all_countries_soy_crush_query = {
    'metric_id': 1800032, # metric ID for crushing (mass)
    'item_id': 270, # item ID for soybeans
    'region_id': [1215, 1029, 1231, 1010], # region IDs for US, Brazil, China, and Argentina
    'source_id': 14, # source ID for USDA PS&D
    'frequency_id': 9, # frequency ID for annual
    'unit_id': 20, # unit ID for bushels
    'start_date': '2022-01-01',
    'end_date': '2022-12-31'
}

all_countries_soy_crush = pd.DataFrame(client.get_data_points(**all_countries_soy_crush_query))
all_countries_soy_crush

,start_date,end_date,value,unit_id,metadata,input_unit_id,input_unit_scale,reporting_date,available_date,metric_id,item_id,region_id,partner_region_id,frequency_id
0,2021-10-01T00:00:00.000Z,2022-09-30T00:00:00.000Z,1.469760e+09,20,{},14,1,2022-03-09T00:00:00.000Z,2022-03-09T00:00:00.000Z,1800032,270,1010,0,9
1,2021-10-01T00:00:00.000Z,2022-09-30T00:00:00.000Z,1.699410e+09,20,{},14,1,2022-03-09T00:00:00.000Z,2022-03-09T00:00:00.000Z,1800032,270,1029,0,9
2,2021-09-01T00:00:00.000Z,2022-08-31T00:00:00.000Z,2.215002e+09,20,{},14,1,2022-03-09T00:00:00.000Z,2022-03-09T00:00:00.000Z,1800032,270,1215,0,9
3,2021-10-01T00:00:00.000Z,2022-09-30T00:00:00.000Z,3.380448e+09,20,{},14,1,2022-03-09T00:00:00.000Z,2022-03-09T00:00:00.000Z,1800032,270,1231,0,9
